In [33]:
!pip install tensorflow-probability

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import random
import math

In [2]:
def findVolume(a,b,r,start,end):

  def integral_l(y, x):
      dx = (x[-1] - x[0]) / (int(x.shape[0]) - 1)
      return tf.reduce_sum(y[:-1])*dx

  def integral_r(y, x):
      dx = (x[-1] - x[0]) / (int(x.shape[0]) - 1)
      return tf.reduce_sum(y[1:])*dx

  start = tf.constant(0, dtype=tf.float32)
  end = tf.constant(2, dtype=tf.float32)

  x = tf.linspace(start, end, 10000)
  result_l = integral_l(b+tf.math.sqrt(r**2-(x-a)**2), x)
  result_r = integral_r(b+tf.math.sqrt(r**2-(x-a)**2), x)

  result = (result_l+result_r)/2

  return result


In [3]:
print(findVolume(0,0,4,-4,4).numpy().reshape(1,1))

[[7.652892]]


In [4]:
NN = tf.keras.models.Sequential([
    tf.keras.layers.Input((1,)),
    tf.keras.layers.Dense(units = 32, activation = 'tanh'),
    tf.keras.layers.Dense(units = 1)
])

NN.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                64        
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 97
Trainable params: 97
Non-trainable params: 0
_________________________________________________________________


In [5]:
optm = tf.keras.optimizers.Adam(learning_rate = 0.001)

In [6]:
def ode_system(t, net):
    t = t.reshape(-1,1)
    t = tf.constant(t, dtype = tf.float32)
    t_0 = tf.zeros((1,1))
    initial_a=tf.fill([1,1],-0.00009899662236288189 )
    vl = tf.fill([1,1],-0.0042040714285714285)
    vr = tf.fill([1,1],0.004339750000000001)
    d = tf.fill([1,1],0.321/1000)
    gap_h = tf.fill([1,1],-0.000091020)

    with tf.GradientTape() as tape:
        tape.watch(t)

        u = net(t)
        a = u
        xl = tf.fill([1,1],0)
        xr = d
        yl = vl*t
        yr = vr*t  - gap_h
        b = (yl+yr)/2-(d/(yr-yl))*(a-d/2)
        r = tf.math.sqrt((0-a)**2+(yl-b)**2)
        A = findVolume(a,b,r,0,d).numpy().reshape(1,1)-d*b
        A_t = tape.gradient(A, t)

    ode_loss = A_t
    IC_loss = net(t_0) - initial_a

    square_loss = tf.square(ode_loss) + tf.square(IC_loss)
    total_loss = tf.reduce_mean(square_loss)

    return total_loss

In [ ]:
train_t = (np.random.rand(10000)).reshape(-1, 1)*0.0282
train_loss_record = []

for itr in range(6000):
    with tf.GradientTape() as tape:
        train_loss = ode_system(train_t, NN)
        train_loss_record.append(train_loss)

        grad_w = tape.gradient(train_loss, NN.trainable_variables)
        optm.apply_gradients(zip(grad_w, NN.trainable_variables))

    if itr % 1000 == 0:
        print(train_loss.numpy())

plt.figure(figsize = (10,8))
plt.plot(train_loss_record)
plt.show()

2.151204e-08


In [63]:
print(NN.predict((0.0280,)).ravel())

1/1 [==============================] - 0s 187ms/step
[0.00241981]


In [9]:
train_t = (np.random.rand(10)).reshape(-1, 1)*5
print(train_t)

[[0.39446693]
 [3.54736698]
 [4.83135874]
 [2.366196  ]
 [0.86753247]
 [3.02531348]
 [3.51018706]
 [3.86727549]
 [3.23200333]
 [1.51407343]]


In [ ]:
for test_t in range(0, 5) :
    test_t=(test_t/100)*(5)
    pred_u = NN.predict((test_t,)).ravel()

    vl = -0.5
    vr = 0.6
    d = 4.
    gap_h = 2.

    a = pred_u
    xl = 0
    xr = d
    yl = vl*test_t
    yr = vr*test_t  - gap_h
    b = (yl+yr)/2-(-d/(yr-yl))*(d/2-a)
    r = tf.math.sqrt((0-a)**2+(yl-b)**2)

    # the equation of a circle : (x-a)^2 + (x-b)^2 = r^2

    # x값 범위 설정
    x = np.linspace(0, d, 1000)

    # 원의 방정식
    y1 = np.sqrt(r**2 - (x-a)**2) + b

    # 시각화
    plt.plot(x, y1, color='blue')
    plt.show()

In [9]:
test_t = np.linspace(0, 0.0280, 1000)

1/1 [==============================] - 0s 29ms/step
[-4.000472]


In [20]:
t_0=tf.fill([1,1],-4.)
print(t_0)

tf.Tensor([[-4.]], shape=(1, 1), dtype=float32)


In [19]:
t_0 = tf.zeros((1,1))
print(t_0)

tf.Tensor([[0.]], shape=(1, 1), dtype=float32)


In [21]:
t = -1.1771358e-004
t = '{:.9f}'.format(t)
print(t)
# 0.000030000

-0.000117714


In [67]:
t =  3.0493201e-005
type(t)
t = '{:.9f}'.format(t)
print(t)
# 0.000030000

0.000030493


In [23]:
t =  -9.1020329e-005
t = '{:.9f}'.format(t)
print(t)
# 0.000030000

-0.000091020


In [24]:
vr =-0.000117714 / 0.0280
print(vr)

-0.0042040714285714285


In [27]:
vl =(0.000030493+0.000091020 )/ 0.0280
print(vl)

0.004339750000000001


In [30]:
t = 1.0874056e+000
thetar10 = '{:.9f}'.format(t)
print(t)

1.0874056


In [31]:
t =   1.6406095e+000
thetar10 = '{:.9f}'.format(t)
print(t)

1.6406095


In [ ]:
vl = -0.0042040714285714285
vr = 0.004339750000000001
d = 0.321/1000
gap_h = -0.000091020
b = (yl+yr)/2-(-d/(yr-yl))*(d/2-a)
r = tf.math.sqrt((0-a)**2+(yl-b)**2)
print(b)
print(r)

In [43]:
x1 = 0.
x2 = 0.321/1000
y1 = 0.0
y2 = -0.000091020
t1 = -0.06992681194
t2 = -0.5317094317

In [47]:
A = np.array([[1,1],[t1,t2]])
B =np.array([x1+t1*y1,x2+t2*y2])
C = np.linalg.solve(A,B)
print(C)


[ 0.00079994 -0.00079994]


In [6]:
print(tf.fill([1,1],5))

tf.Tensor([[5]], shape=(1, 1), dtype=int32)


In [5]:
print(tf.zeros((1,1)))

tf.Tensor([[0.]], shape=(1, 1), dtype=float32)


In [ ]:
tf.math.sq